In [1]:
import sys
import torch

In [2]:
GAZETA_PATH = '../data/gazeta_jsonl'

In [3]:
import json

In [4]:
def read_gazeta_records(file_name, shuffle=False, sort_by_date=True):
    assert shuffle != sort_by_date
    records = []
    with open(file_name, "r") as r:
        for line in r:
            records.append(json.loads(line))
    if sort_by_date:
        records.sort(key=lambda x: x["date"])
    if shuffle:
        random.shuffle(records)
    return records

In [5]:
import os

In [6]:
dataset_files = {
    'train': os.path.join(GAZETA_PATH,'gazeta_train.jsonl'),
    'val': os.path.join(GAZETA_PATH,'gazeta_val.jsonl'),
    'test': os.path.join(GAZETA_PATH, 'gazeta_test.jsonl')
}

In [7]:
records = {
    split: read_gazeta_records(path) for split, path in dataset_files.items()
}

In [8]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [9]:
from GPTSummarizer import GPTSummarizerPL

Предварительно нужно обучить модель

In [10]:
model = GPTSummarizerPL.load_from_checkpoint('gpt_checkpoint_gazeta3/epoch=1-step=6549.ckpt')

In [11]:
tokenizer = GPT2Tokenizer.from_pretrained('sberbank-ai/rugpt3medium_based_on_gpt2')

In [12]:
from rouge import Rouge

In [13]:
def extract_summary(model, text, max_input_length, max_output_length, **generate_args):
    with torch.no_grad():
        vocab=tokenizer.get_vocab()
        bos_token_id = vocab['<s>']
        eos_token_id = vocab['</s>']
        pad_token_id = vocab['<pad>']
        encoded_text = [bos_token_id] +\
            tokenizer.encode(text)[:max_input_length] + [eos_token_id]
        encoded_text = torch.tensor(encoded_text, device=get_model_device(model)).view(1,-1)
#         print(encoded_text.shape)
        encoded_output = model.gpt.generate(encoded_text,
                                            bos_token_id=bos_token_id,
                                            eos_token_id=eos_token_id,
                                            pad_token_id=pad_token_id,
                                            max_length=max_input_length + max_output_length + 2,
                                            **generate_args)

        indices = encoded_output[0].tolist()

        first_eos_index = indices.index(eos_token_id)
        sum_start_index = first_eos_index + 1

        final_indices = []
#         print(indices[first_eos_index:])
        final_indices = indices[sum_start_index:-1]
#         for idx in indices[sum_start_index:]:
#             if idx != eos_token_id:
#                 final_indices.append(idx)
#             else:
#                 break
    return tokenizer.decode(final_indices)

In [14]:
def calc_metrics(refs, hyps, metric="all"):
    metrics = dict()
    metrics["count"] = len(hyps)
    metrics["ref_example"] = refs[-1]
    metrics["hyp_example"] = hyps[-1]

    if metric in ("rouge", "all"):
        rouge = Rouge()
        scores = rouge.get_scores(hyps, refs, avg=True)
        metrics.update(scores)

    return metrics

In [15]:
def print_metrics(refs, hyps, metric="all"):
    metrics = calc_metrics(refs, hyps, metric=metric)

    print("-------------METRICS-------------")
    print("Count:\t", metrics["count"])
    print("Ref:\t", metrics["ref_example"])
    print("Hyp:\t", metrics["hyp_example"])

#     if "bleu" in metrics:
#         print("BLEU:     \t{:3.1f}".format(metrics["bleu"] * 100.0))
    if "rouge-1" in metrics:
#         print([metrics["rouge-1"][m] * 100.0 for m in ('p','r','f')])
        print("ROUGE-1: P: {:3.2f} R: {:3.2f} F: {:3.2f}".format(
            *[metrics["rouge-1"][m] * 100.0 for m in ['p','r','f']]))
        print("ROUGE-2: P: {:3.2f} R: {:3.2f} F: {:3.2f}".format(
            *[metrics["rouge-2"][m] * 100.0 for m in ['p','r','f']]))
        print("ROUGE-L: P: {:3.2f} R: {:3.2f} F: {:3.2f}".format(
            *[metrics["rouge-l"][m] * 100.0 for m in ['p','r','f']]))


In [16]:
import razdel

In [17]:
def postprocess(refs, hyps, tokenize_after=True, lower=True):
    for i, (ref, hyp) in enumerate(zip(refs, hyps)):
        ref = ref.strip()
        hyp = hyp.strip()
        if tokenize_after:
            hyp = " ".join([token.text for token in razdel.tokenize(hyp)])
            ref = " ".join([token.text for token in razdel.tokenize(ref)])
        if lower:
            hyp = hyp.lower()
            ref = ref.lower()
        refs[i] = ref
        hyps[i] = hyp
    return refs, hyps

In [18]:
from tqdm.notebook import tqdm

In [19]:
def get_model_device(model):
    return next(iter(model.parameters())).device

In [20]:
def calc_method_score(records, predict_func, nrows=None, return_ref_pred=False, text_key='text'):
    references = []
    predictions = []

    for i, record in tqdm(enumerate(records)):
        if nrows is not None and i >= nrows:
            break
        summary = record["summary"]
        text = record[text_key]
        prediction = predict_func(text, summary)
        references.append(summary)
        predictions.append(prediction)
    references, predictions = postprocess(references, predictions)
    print_metrics(references, predictions)
    if return_ref_pred:
        return references, predictions

In [21]:
def predict_with_gpt(text, summary):
    summary = extract_summary(model, text, 601, 163,
            no_repeat_ngram_size=3,
            num_beams=10,
            top_k=0,
            early_stopping=True)
    return summary

In [22]:
model = model.cuda()

In [23]:
import random

In [24]:
len(records['test'])

5770

In [25]:
with torch.no_grad():
    rand_index = random.randrange(len(records['val']))
    rand_index = 3534
    print(rand_index)
    text = records['val'][rand_index]['text']
    print(text)
    print("-----------")
    ref = records['val'][rand_index]['summary']
    print("Reference\n")
    
    print(ref)
    print("\nGenerated\n")
    
    print(extract_summary(model,text, 600,163,
            no_repeat_ngram_size=3,
            num_beams=10,
            top_k=0,
            early_stopping=True))

3534
Отношения Германии и США переживают не лучшие времена, разногласия по политическим и экономическим вопросам и угрозы со стороны Вашингтона оказали негативный эффект на взаимодействие двух стран. Об этом пишет журнал Der Spiegel. По данным аналитиков издания, в настоящее время 85% граждан ФРГ негативно или резко негативно относятся к США, а 42% считают ключевым партнером Китай. Одной из причин ухудшения отношений Берлина с Вашингтоном стало назначение посла США в ФРГ Ричарда Гренелла. Дипломат занял данный пост в мае 2018 года, и с тех пор «стороны играют в молчанку». Немецкие чиновники постепенно начали избегать любых встреч с Гренеллом , в частности, канцлер Германии Ангела Меркель ни разу с ним не общалась, пишет журнал. Власти ФРГ начали игнорировать посла США из-за его поведения. К примеру, дипломат фактически оборвал контакты с организацией «Атлантический мост», которая является ключевой в диалоге Берлина и Вашингтона. Также Гренелл не раз выступал с критикой в адрес правител

In [26]:
model = model.eval()

In [27]:
random.seed(4543)

In [28]:
refs, preds = calc_method_score(random.sample(records['val'],70), predict_with_gpt, return_ref_pred=True)

|          | 0/? [00:00<?, ?it/s]

-------------METRICS-------------
Count:	 70
Ref:	 украинская певица светлана лобода пожаловалась подписчикам на жуткие гематомы . согласно артистке , травмы стали следствием ее концертной деятельности .
Hyp:	 певица света лобода рассказала о травмах , полученных во время выступлений на сцене . согласно артистке , в 2011 году она выступала с концертами в москве , нижнем новгороде и других городах .
ROUGE-1: P: 29.29 R: 20.34 F: 23.29
ROUGE-2: P: 6.30 R: 4.18 F: 4.85
ROUGE-L: P: 25.44 R: 17.51 F: 20.29


In [29]:
with open('gpt_result.txt', 'w+') as f:
    for ref, hyp in zip(refs, preds):
        f.write(ref)
        f.write("\n\n")
        f.write(hyp)
        f.write("\n\n=============\n\n")